In [1]:
import numpy as np # linear algebra
from scipy.stats.stats import pearsonr
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook as tqdm
import seaborn as sns 
import matplotlib.pyplot as plt
import math
sns.set()
import os
import openbabel as ob
pd.set_option('display.max_columns', 500)

In [36]:
train = pd.read_csv('train.csv')
test = pd.read_csv(f'test.csv')
structures = pd.read_csv('structures.csv')
x = structures.groupby('molecule_name').atom_index.max().reset_index(drop=False)
x.columns = ['molecule_name','totalatoms']
x.totalatoms+=1
train = train.merge(x,on='molecule_name')
test = test.merge(x,on='molecule_name')
#train = train[train.molecule_name=='dsgdb9nsd_000001']

In [37]:
obConversion = ob.OBConversion()
obConversion.SetInFormat("xyz")
structdir='./Structures/'
mols=[]
mols_files=os.listdir(structdir)
mols_index=dict(map(reversed,enumerate(mols_files)))
for f in mols_index.keys():
    try:
        mol = ob.OBMol()
        obConversion.ReadFile(mol, structdir+f) 
        mols.append(mol)
    except:
        print("Wrong format")
        pass

In [38]:
# Get atom information for train
methods = [ "eem", "mmff94", "gasteiger", "qeq", "qtpie", 
               "eem2015ha", "eem2015hm", "eem2015hn", "eem2015ba", "eem2015bm", "eem2015bn" ]
def create_dipole_moments(df, mols):
    stats = []
    for m, groupdf in tqdm(df.groupby('molecule_name')):
        mol=mols[mols_index[m+'.xyz']]
        entrystats = {}
        entrystats['molecule_name'] = m
        for method in methods:
            # on the command-line, you can get partial charges like this
            # obabel CHF3.g03 -onul --partialcharge gasteiger --print
            cm = ob.OBChargeModel_FindType(method)
            cm.ComputeCharges(mol)
            # but in C++ or Python we can get the dipole moment vector
            dipole = cm.GetDipoleMoment(mol)
            entrystats[f'dipole_{method}_x'] = cm.GetDipoleMoment(mol).GetX()
            entrystats[f'dipole_{method}_y'] = cm.GetDipoleMoment(mol).GetY()
            entrystats[f'dipole_{method}_z'] = cm.GetDipoleMoment(mol).GetZ()         
            entrystats[f'moment_{method}'] = moment = math.sqrt(dipole.GetX()**2+dipole.GetY()**2+dipole.GetZ()**2)
            
        stats.append(entrystats)
        
    outfile = pd.DataFrame(stats)    
    return outfile
        #entrystats = {}
        #entrystats['molecule_name'] = m
        #entrystats['molecule_dipole_moment_x'] = openbabel.OBChargeModel.GetDipoleMoment(mol)[0]
        #entrystats['molecule_dipole_moment_y'] = openbabel.OBChargeModel.GetDipoleMoment(mol)[1]
        #entrystats['molecule_dipole_moment_z'] = openbabel.OBChargeModel.GetDipoleMoment(mol)[2]

In [39]:
dipole_train = create_dipole_moments(train, mols)

In [42]:
dipole_train.shape

(85003, 45)

In [43]:
dipole_test = create_dipole_moments(test, mols)

In [44]:
dipole_test.head()

,dipole_eem2015ba_x,dipole_eem2015ba_y,dipole_eem2015ba_z,dipole_eem2015bm_x,dipole_eem2015bm_y,dipole_eem2015bm_z,dipole_eem2015bn_x,dipole_eem2015bn_y,dipole_eem2015bn_z,dipole_eem2015ha_x,...,moment_eem2015ba,moment_eem2015bm,moment_eem2015bn,moment_eem2015ha,moment_eem2015hm,moment_eem2015hn,moment_gasteiger,moment_mmff94,moment_qeq,moment_qtpie
0,1.831868e-15,0.000000,-2.775558e-17,-1.743050e-14,0.000000,0.000000,1.124101e-14,0.000000,2.775558e-17,-6.106227e-16,...,1.832078e-15,1.743050e-14,1.124104e-14,6.106227e-16,6.106227e-16,2.581269e-15,0.000000,0.000000,8.716683e-16,1.121603e-16
1,2.767103e-01,0.378973,-4.974225e-01,2.390778e-01,0.327434,-0.429776,2.512039e-01,0.344042,-4.515744e-01,3.190803e-01,...,6.838260e-01,5.908292e-01,6.207960e-01,7.885332e-01,7.043739e-01,6.488648e-01,1.795077,1.687692,9.654060e-01,1.005163e+00
2,7.288001e-06,-0.000010,-1.032895e-07,9.306704e-05,-0.000132,-0.000001,8.646845e-05,-0.000119,-1.226044e-06,-1.644540e-06,...,1.242947e-05,1.612929e-04,1.472976e-04,2.820404e-06,1.574348e-04,1.340853e-04,0.000116,0.000359,2.170352e-04,3.597267e-04
3,8.953699e-01,0.533465,-5.753728e-03,4.656249e-01,0.277433,-0.003010,8.072865e-01,0.480979,-5.218250e-03,9.406552e-01,...,1.042260e+00,5.420191e-01,9.397232e-01,1.094976e+00,6.428395e-01,9.980073e-01,4.090892,3.342608,2.067661e+00,2.559191e+00
4,7.068226e-01,0.526501,-6.808330e-02,5.732893e-01,0.426672,-0.057323,6.128178e-01,0.466284,-8.003477e-02,8.605125e-01,...,8.839888e-01,7.169346e-01,7.741912e-01,1.057254e+00,9.024214e-01,8.075849e-01,1.493993,1.780029,1.123998e+00,1.488473e+00


In [45]:
dipole_train.to_csv('df_dipole_train.csv', index = False)
dipole_test.to_csv('df_dipole_test.csv', index = False)

In [2]:
dipole_train = pd.read_csv('df_dipole_train.csv')
dipole_test = pd.read_csv('df_dipole_test.csv')

In [6]:
dipole_train.head()

,dipole_eem2015ba_x,dipole_eem2015ba_y,dipole_eem2015ba_z,dipole_eem2015bm_x,dipole_eem2015bm_y,dipole_eem2015bm_z,dipole_eem2015bn_x,dipole_eem2015bn_y,dipole_eem2015bn_z,dipole_eem2015ha_x,dipole_eem2015ha_y,dipole_eem2015ha_z,dipole_eem2015hm_x,dipole_eem2015hm_y,dipole_eem2015hm_z,dipole_eem2015hn_x,dipole_eem2015hn_y,dipole_eem2015hn_z,dipole_eem_x,dipole_eem_y,dipole_eem_z,dipole_gasteiger_x,dipole_gasteiger_y,dipole_gasteiger_z,dipole_mmff94_x,dipole_mmff94_y,dipole_mmff94_z,dipole_qeq_x,dipole_qeq_y,dipole_qeq_z,dipole_qtpie_x,dipole_qtpie_y,dipole_qtpie_z,molecule_name,moment_eem,moment_eem2015ba,moment_eem2015bm,moment_eem2015bn,moment_eem2015ha,moment_eem2015hm,moment_eem2015hn,moment_gasteiger,moment_mmff94,moment_qeq,moment_qtpie
0,5.257599e-07,-4.171933e-07,9.587587e-09,0.000007,-0.000006,2.250240e-07,0.000007,-0.000006,1.643369e-07,-1.185248e-07,9.461598e-08,-2.638400e-09,0.000007,-0.000006,2.338656e-07,0.000006,-0.000005,1.505996e-07,5.162559e-06,-0.000004,1.080148e-07,3.011039e-06,-0.000003,1.529568e-07,0.000000,0.000000,0.000000,-0.000029,2.384273e-05,-9.043553e-07,0.000057,-0.000046,1.664848e-06,dsgdb9nsd_000001,0.000007,6.712419e-07,0.000009,0.000009,1.516815e-07,0.000009,0.000008,0.000004,0.000000,0.000038,0.000073
1,1.718592e-01,-1.150444e-01,-3.280418e-01,0.173404,-0.116079,-3.309895e-01,0.152517,-0.102097,-2.911221e-01,1.938536e-01,-1.297679e-01,-3.700241e-01,0.194019,-0.129879,-3.703379e-01,0.158096,-0.105831,-3.017706e-01,1.658959e-01,-0.111053,-3.166591e-01,2.198626e-01,-0.146748,-4.183036e-01,0.741305,-0.496242,-1.414974,-0.749928,5.020120e-01,1.431667e+00,1.387155,-0.928579,-2.648194e+00,dsgdb9nsd_000002,0.374336,3.877915e-01,0.391277,0.344147,4.374206e-01,0.437792,0.356735,0.509807,1.672704,1.692359,3.130397
2,4.666781e-01,-2.941880e-01,-6.023551e-03,0.346085,-0.218167,-4.467017e-03,0.422960,-0.266629,-5.459272e-03,4.950977e-01,-3.121034e-01,-6.390371e-03,0.404522,-0.255005,-5.221278e-03,0.432155,-0.272425,-5.577954e-03,3.200435e-01,-0.201751,-4.130896e-03,8.190548e-01,-0.508684,-1.037144e-02,1.666745,-1.050695,-0.021513,-1.715619,1.081504e+00,2.214399e-02,2.885833,-1.819193,-3.724829e-02,dsgdb9nsd_000003,0.378350,5.516985e-01,0.409135,0.500016,5.852957e-01,0.478218,0.510886,1.019033,1.970396,2.028174,3.411581
3,-1.297169e-02,9.554743e-01,5.266079e-03,-0.004135,0.304562,1.678588e-03,-0.005773,0.425199,2.343475e-03,-1.597095e-02,1.176395e+00,6.483680e-03,-0.005771,0.425080,2.342819e-03,-0.006056,0.446091,2.458624e-03,-6.489974e-03,0.478041,2.634716e-03,-4.744895e-02,3.580260,2.003849e-02,-0.043466,3.201615,0.017646,0.012270,-9.038000e-01,-4.981276e-03,-0.011307,0.832866,4.590326e-03,dsgdb9nsd_000005,0.478093,9.555769e-01,0.304595,0.425244,1.176521e+00,0.425125,0.446139,3.368599,3.201959,0.903897,0.832956
4,-1.057558e-07,1.215925e-07,4.084135e-08,-0.000001,0.000002,6.015577e-07,-0.000001,0.000002,5.300113e-07,2.367787e-08,-2.864750e-08,-9.528736e-09,-0.000001,0.000002,6.020938e-07,-0.000001,0.000001,4.844108e-07,-9.445143e-07,0.000001,3.748800e-07,-6.343331e-07,0.000001,3.580509e-07,0.000000,0.000000,0.000000,0.000003,4.681395e-07,-5.735078e-07,-0.000005,-0.000001,9.073841e-07,dsgdb9nsd_000007,0.000002,1.662439e-07,0.000002,0.000002,3.836819e-08,0.000002,0.000002,0.000001,0.000000,0.000003,0.000005


In [5]:
methods = [ "eem", "mmff94", "gasteiger", "qeq", "qtpie", 
               "eem2015ha", "eem2015hm", "eem2015hn", "eem2015ba", "eem2015bm", "eem2015bn" ]

In [9]:
def dip_magn(df, methods):
    for m in methods:
        t_p_0 = df[[f'dipole_{m}_x', f'dipole_{m}_y', f'dipole_{m}_z']].values

        df[f'dipole_{m}_magnitude'] = np.linalg.norm(t_p_0, axis=1)
    return df


In [10]:
dipole_train = dip_magn(dipole_train, methods)

In [12]:
dipole_test = dip_magn(dipole_test, methods)

In [13]:
dipole_test.head()

,dipole_eem2015ba_x,dipole_eem2015ba_y,dipole_eem2015ba_z,dipole_eem2015bm_x,dipole_eem2015bm_y,dipole_eem2015bm_z,dipole_eem2015bn_x,dipole_eem2015bn_y,dipole_eem2015bn_z,dipole_eem2015ha_x,dipole_eem2015ha_y,dipole_eem2015ha_z,dipole_eem2015hm_x,dipole_eem2015hm_y,dipole_eem2015hm_z,dipole_eem2015hn_x,dipole_eem2015hn_y,dipole_eem2015hn_z,dipole_eem_x,dipole_eem_y,dipole_eem_z,dipole_gasteiger_x,dipole_gasteiger_y,dipole_gasteiger_z,dipole_mmff94_x,dipole_mmff94_y,dipole_mmff94_z,dipole_qeq_x,dipole_qeq_y,dipole_qeq_z,dipole_qtpie_x,dipole_qtpie_y,dipole_qtpie_z,molecule_name,moment_eem,moment_eem2015ba,moment_eem2015bm,moment_eem2015bn,moment_eem2015ha,moment_eem2015hm,moment_eem2015hn,moment_gasteiger,moment_mmff94,moment_qeq,moment_qtpie,dipole_eem_magnitude,dipole_mmff94_magnitude,dipole_gasteiger_magnitude,dipole_qeq_magnitude,dipole_qtpie_magnitude,dipole_eem2015ha_magnitude,dipole_eem2015hm_magnitude,dipole_eem2015hn_magnitude,dipole_eem2015ba_magnitude,dipole_eem2015bm_magnitude,dipole_eem2015bn_magnitude
0,1.831868e-15,0.000000,-2.775558e-17,-1.743050e-14,0.000000,0.000000,1.124101e-14,0.000000,2.775558e-17,-6.106227e-16,0.000000,0.000000e+00,6.106227e-16,0.000000,0.000000,-2.581269e-15,0.000000,0.000000,-2.775558e-16,0.000000,2.775558e-17,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,-8.699708e-16,0.000000,5.437317e-17,-1.121603e-16,0.000000,0.000000,dsgdb9nsd_000004,2.789401e-16,1.832078e-15,1.743050e-14,1.124104e-14,6.106227e-16,6.106227e-16,2.581269e-15,0.000000,0.000000,8.716683e-16,1.121603e-16,2.789401e-16,0.000000,0.000000,8.716683e-16,1.121603e-16,6.106227e-16,6.106227e-16,2.581269e-15,1.832078e-15,1.743050e-14,1.124104e-14
1,2.767103e-01,0.378973,-4.974225e-01,2.390778e-01,0.327434,-0.429776,2.512039e-01,0.344042,-4.515744e-01,3.190803e-01,0.437001,-5.735877e-01,2.850238e-01,0.390360,-0.512370,2.625622e-01,0.359597,-0.471992,2.070955e-01,0.283632,-3.722830e-01,0.738829,1.018077,-1.339192e+00,0.682925,0.935309,-1.227645,-3.906487e-01,-0.535023,7.022485e-01,4.067337e-01,0.557056,-0.731170,dsgdb9nsd_000015,5.117912e-01,6.838260e-01,5.908292e-01,6.207960e-01,7.885332e-01,7.043739e-01,6.488648e-01,1.795077,1.687692,9.654060e-01,1.005163e+00,5.117912e-01,1.687692,1.837330,9.654060e-01,1.005163e+00,7.885332e-01,7.043739e-01,6.488648e-01,6.838260e-01,5.908292e-01,6.207960e-01
2,7.288001e-06,-0.000010,-1.032895e-07,9.306704e-05,-0.000132,-0.000001,8.646845e-05,-0.000119,-1.226044e-06,-1.644540e-06,0.000002,2.342386e-08,9.112539e-05,-0.000128,-0.000001,7.879315e-05,-0.000108,-0.000001,6.183242e-05,-0.000085,-8.729611e-07,0.000055,-0.000094,-8.727754e-07,0.000181,-0.000310,-0.000003,-1.417779e-04,0.000164,1.908176e-06,2.422749e-04,-0.000266,-0.000003,dsgdb9nsd_000016,1.047960e-04,1.242947e-05,1.612929e-04,1.472976e-04,2.820404e-06,1.574348e-04,1.340853e-04,0.000116,0.000359,2.170352e-04,3.597267e-04,1.047960e-04,0.000359,0.000109,2.170352e-04,3.597267e-04,2.820404e-06,1.574348e-04,1.340853e-04,1.242947e-05,1.612929e-04,1.472976e-04
3,8.953699e-01,0.533465,-5.753728e-03,4.656249e-01,0.277433,-0.003010,8.072865e-01,0.480979,-5.218250e-03,9.406552e-01,0.560449,-6.033535e-03,5.522358e-01,0.329037,-0.003556,8.573564e-01,0.510811,-0.005532,5.435602e-01,0.323863,-3.508873e-03,3.694611,2.276731,-2.536270e-02,2.871513,1.710877,-0.018582,-1.776195e+00,-1.058407,1.126997e-02,2.198376e+00,1.310120,-0.013799,dsgdb9nsd_000020,6.327378e-01,1.042260e+00,5.420191e-01,9.397232e-01,1.094976e+00,6.428395e-01,9.980073e-01,4.090892,3.342608,2.067661e+00,2.559191e+00,6.327378e-01,3.342608,4.339850,2.067661e+00,2.559191e+00,1.094976e+00,6.428395e-01,9.980073e-01,1.042260e+00,5.420191e-01,9.397232e-01
4,7.068226e-01,0.526501,-6.808330e-02,5.732893e-01,0.426672,-0.057323,6.128178e-01,0.466284,-8.003477e-02,8.605125e-01,0.611990,-5.265603e-02,7.340641e-01,0.521280,-0.061496,6.424018e-01,0.482808,-0.080061,4.525469e-01,0.348833,-6.447504e-02,1.106591,1.128717,-2.829856e-01,0.676756,1.135692,-1.191935,-5.125530e-01,-0.931076,3.657319

In [14]:
dipole_train.to_csv('df_dipole_train.csv', index = False)
dipole_test.to_csv('df_dipole_test.csv', index = False)